In [1]:
 # Put these at the top of every notebook, to get automatic reloading and inline plotting
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import sys
sys.path.append('/home/ubuntu/fastai/')
from fastai.imports import *
from fastai.transforms import *
from fastai.conv_learner import *
from fastai.model import *
from fastai.dataset import *
from fastai.sgdr import *
from fastai.plots import *
from glob2 import glob
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix

In [3]:
f_model = resnet50
PATH = '/home/ubuntu/plants/Image-processing/'
label_csv ='labels.csv'
n = len(list(open(label_csv)))-1

In [4]:
def f1(preds, targs):
    preds = np.argmax(preds, 1)
    return f1_score(targs, preds, average='micro')

In [5]:
def get_data(sz):
    tfms = tfms_from_model(f_model, sz, max_zoom=1.05)
    return ImageClassifierData.from_csv(PATH, 'train', label_csv, tfms=tfms,
                    val_idxs=[0], test_name='test')

In [6]:
data = get_data(256)

In [7]:
learn = ConvLearner.pretrained(f_model,data, ps=0.5,metrics=[f1],precompute= False)

In [8]:
learn.load('256')

In [9]:
lr =1e-2
lrs =np.array([lr/18,lr/6,lr/2])

In [10]:
learn.freeze()
learn.fit(lr, 3, cycle_len=1, cycle_mult=2)
learn.unfreeze()
learn.fit(lrs, 3, cycle_len=1, cycle_mult=2)

HBox(children=(IntProgress(value=0, description='Epoch', max=7), HTML(value='')))

[ 0.       0.0895   0.00808  1.     ]                       
[ 1.       0.08473  0.00448  1.     ]                       
[ 2.       0.07905  0.00543  1.     ]                       
[ 3.       0.0732   0.00549  1.     ]                       
[ 4.       0.06711  0.00529  1.     ]                       
[ 5.       0.06172  0.00333  1.     ]                       
[ 6.       0.06778  0.00374  1.     ]                       



HBox(children=(IntProgress(value=0, description='Epoch', max=7), HTML(value='')))

[ 0.       0.08094  0.00635  1.     ]                       
[ 1.       0.07601  0.00168  1.     ]                       
[ 2.       0.0475   0.00214  1.     ]                       
[ 3.       0.03769  0.00434  1.     ]                       
[ 4.       0.03063  0.00069  1.     ]                       
[ 5.       0.02156  0.00089  1.     ]                       
[ 6.       0.02265  0.00053  1.     ]                       



In [11]:
pred,y = learn.TTA()
preds = np.mean(np.exp(pred),0)
f1(preds,y)

1.0

In [12]:
submission, test_y = learn.TTA(is_test=True)

In [13]:
submission = np.mean(np.exp(submission),0)
submission = np.argmax(submission,axis=1)

In [14]:
func = np.vectorize(lambda x:data.classes[x])
species = func(submission)
test_df = [f.split('/')[1] for f in data.test_dl.dataset.fnames]
submission_df = pd.DataFrame({'file': test_df, 'species': species})
submission_df.species.replace('_',' ',regex=True,inplace =True)
submission_df.to_csv('submission.csv.gz', compression = 'gzip',index=False)